# Instructor Do: Intro to VADER Sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ddevii/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]



Total articles about Facebook Libra: 40


{'source': {'id': 'axios', 'name': 'Axios'},
 'author': 'Scott Rosenberg',
 'title': 'Facebook keeps pivoting to new projects — and then abandoning them',
 'description': 'Facebook\'s plan to wind down its podcast platform is the latest in a long line of projects the social network has launched with fanfare and then unceremoniously scuttled.Why it matters: "Launch, test, shut down" is a common pattern in tech\'s "fail fast" cultu…',
 'url': 'https://www.axios.com/2022/05/04/facebooks-endless-pivot-metaverse-podcasts',
 'urlToImage': 'https://images.axios.com/9wnl4CnD6xg67vSiqYA-yvPKwbs=/0x0:2545x1432/1366x768/2022/05/04/1651665611715.jpg',
 'publishedAt': '2022-05-04T10:00:32Z',
 'content': 'Facebook\'s plan to wind down its podcast platform is the latest in a long line of projects the social network has launched with fanfare and then unceremoniously scuttled.\r\nWhy it matters: "Launch, te… [+3317 chars]'}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()



,date,text,compound,positive,negative,neutral
0,2022-05-04,Facebook's plan to wind down its podcast platf...,0.1531,0.075,0.000,0.925
1,2022-05-11,The European Commission is considering hard cu...,0.2263,0.064,0.039,0.896
2,2022-05-04,Facebook's plan to wind down its podcast platf...,0.1531,0.075,0.000,0.925
3,2022-05-10,"Thanks to Meta Platforms CEO Mark Zuckerberg, ...",0.2896,0.085,0.000,0.915
4,2022-05-12,"Don't miss CoinDesk's Consensus 2022, the must...",0.7831,0.214,0.000,0.786


In [7]:
# Get descriptive stats from the DataFrame
libra_df.describe()



,compound,positive,negative,neutral
count,40.000000,40.000000,40.000000,40.000000
mean,0.224625,0.093375,0.038600,0.868075
std,0.472168,0.078172,0.059406,0.086675
min,-0.848100,0.000000,0.000000,0.699000
25%,0.000000,0.000000,0.000000,0.799500
50%,0.292800,0.083000,0.000000,0.872500
75%,0.617850,0.156500,0.073000,0.919750
max,0.783100,0.225000,0.221000,1.000000
